In [2]:
from __future__ import annotations
from typing import Any, List, Dict, Set, Tuple, Optional

# 📖 Tema 2 - Algorismes numèrics

### 3.9 Primeritat i el Teorema de Fermat

### ✍️ Pregunta 1 — Primers de **Wieferich** (base 2)

**Definició.**  
Un *primer de Wieferich* és un nombre primer $p$ tal que:

$$
2^{\,p-1} \equiv 1 \pmod{p^2}
$$

És a dir, $p^2$ divideix $2^{\,p-1} - 1$.

---

**Tasca.**  
Troba **tots els primers de Wieferich menors que 5000**.

---

In [3]:
#Solució

def eratostenes(n: int) -> set[int]:
    """
    Retorna el conjunt de tots els nombres primers menors que n,
    emprant una versió senzilla de la criba d'Eratòstenes amb conjunts.
    """
    if n < 2:
        # Si n és menor que 2, no hi ha nombres primers
        return set()
    
    # Inicialitzem el conjunt amb tots els nombres de 2 fins a n-1
    # Cost: O(n) en temps i O(n) en memòria
    nombres = set(range(2, n))

    # Iterem pels possibles factors fins a l’arrel quadrada de n
    # Nombre d'iteracions ≈ √n
    for p in range(2, int(n**0.5) + 1):
        # Comprovem si p encara és al conjunt (és primer)
        # Operació sobre set → O(1)
        if p in nombres:
            # Generem tots els múltiples de p (2p, 3p, 4p, ...)
            # Nombre d’elements ≈ n/p → Cost O(n/p)
            multiples = set(range(p*2, n, p))
            
            # Eliminem els múltiples del conjunt
            # Cost proporcional a la mida del conjunt eliminat: O(n/p)
            nombres -= multiples

    # Retornem el conjunt restant de nombres primers
    return nombres

    # -------------------------------
    # Complexitat temporal: O(n log log n)
    # Complexitat espacial: O(n)
    # -------------------------------



def primers_Wieferich(n = 5000):
    """
    Calcula els primers de Wieferich fins a n (per defecte 5000).
    """
    # Llista on desarem els primers de Wieferich trobats
    wieferich = []

    # Obtenim tots els primers menors que n
    # Cost: O(n log log n)
    for p in eratostenes(n):
        # Comprovem la condició 2^(p-1) ≡ 1 (mod p^2)
        # La funció pow() fa l’exponenciació modular amb cost O(log p)
        if pow(2, p - 1, p**2) == 1:
            # Si la condició és certa, afegim p a la llista
            wieferich.append(p)

    # Retornem tots els primers de Wieferich trobats
    return wieferich

    # -------------------------------
    # Complexitat temporal: O(n log log n)
    #   - Domina el cost de la criba
    # Complexitat espacial: O(n)
    # -------------------------------


In [4]:
assert primers_Wieferich(n = 5000) == [1093, 3511], "Els únics primers Wieferich fins a 2276306935816522 són 1093 i 3511, però no els estàs retornant correctament"

#### 💡 Pistes

- Verifica primer que $p$ és **primer** (amb el sedàs d’Eratòstenes).
- Per comprovar la condició, evita números gegants usant **exponenciació modular** eficient amb:
  ```python
  pow(2, p - 1, p * p) == 1

#### ⚙️ Exemple d’ús de `pow()` amb mòdul

En Python, la funció incorporada `pow(a, b, m)` calcula de forma eficient  
$$ 
a^b \equiv m
$$  
és a dir, el **residu** de dividir $a^b$ per $m$.

Això és molt útil quan els nombres són grans, ja que Python ho fa amb **exponenciació binària** (complexitat $O(\log b))$ sense haver de calcular $a^b$ complet.

Exemples:

In [6]:
import time

N:int=1000

# Exemple 1 — sense mòdul
t0:float = time.perf_counter()
res1:int = pow(2, N)        # 1024
t1:float = time.perf_counter()
print(f"Exemple 1 → Temps: {t1 - t0:.8f} s")

# Exemple 2 — amb mòdul petit
t0:float= time.perf_counter()
res2:int = pow(2, N, 5)     # 1024 mod 5 = 4
t1:float = time.perf_counter()
print(f"Exemple 2 → Temps: {t1 - t0:.8f} s")

# Exemple 3 — exponent gran, però càlcul ràpid
t0:float = time.perf_counter()
res3:int = pow(2, N, 13)  # retorna 3, car 2^1000 ≡ 3 (mod 13)
t1:float = time.perf_counter()
print(f"Exemple 3 → Temps: {t1 - t0:.8f} s")

Exemple 1 → Temps: 0.00010210 s
Exemple 2 → Temps: 0.00010430 s
Exemple 3 → Temps: 0.00010440 s


## ✍️ Pregunta 2 — Tècniques de factorització i prova de Fermat

En aquesta activitat treballarem dues tècniques diferents per comprovar si un nombre $n$ és **primer**, comparant-ne tant l’enfocament com el **temps d’execució**.

---

### 🔹 Part 1 — Comprovació per factorització directa

Escriu una funció `es_primer_factoritzacio(n)` que determini si un nombre $n$ és primer mitjançant **prova de divisibilitat**.  
El procediment ha de seguir aquests passos:

1. Comprovar els casos trivials ($n < 2$, $n=2$, $n=3$).  
2. Verificar si $n$ és divisible per algun nombre enter $d$ tal que $2 \le d \le \sqrt{n}$.  
3. Si es troba algun divisor, retornar `False`; en cas contrari, retornar `True`.  
4. Fer servir el mòdul `time` per mesurar el **temps d’execució total** de la funció i imprimir-lo.

---

### 🔹 Part 2 — Comprovació per la prova de Fermat

Implementa una segona funció `es_primer_fermat(n)` que comprovi si un nombre $n > 10$ és probablement primer aplicant la **prova de Fermat** per a les bases $a = 2, 3, 5$.

El test de Fermat es basa en el teorema següent:

> Si $p$ és primer i $a$ és un enter tal que $1 < a < p$, aleshores  
> $$
> a^{p-1} \equiv 1 \pmod{p}
> $$

És a dir, $p$ divideix $a^{\,p-1} - 1$.

Per tant, per a cada $a$ en $\{2, 3, 5\}$ fes la comprobació i:  
- Si algun resultat és diferent de $1$, pots concloure que **$n$ és compost**.  
- Si tots els resultats són $1$, llavors **$n$ és probablement primer** (pseudoprim), però no amb garantia absoluta.

També aquí cal imprimir el **temps d’execució** de la comprovació.

---

### ⚖️ Objectius de l’exercici

- Comparar l'**eficiència** i la **precisió** de dues tècniques diferents de comprovació de primeritat.  
- Observar com el **temps d’execució** creix amb valors grans de $n$.  
- Entendre la diferència entre **proves deterministes** (factorització) i **proves probabilístiques** (Fermat).

---

### 🧠 Suggeriment addicional

Prova amb diversos valors de $n$ (per exemple, 12, 53, 1729, 9973, 2265091) i comenta:
- Quina tècnica és més ràpida?
- En quins casos la prova de Fermat pot fallar?

In [7]:
import math
import time


def factorp(N: int) -> bool:
    """
    Comprova si un determinat nombre és primer mitjançant la tècnica de factorització.
    Mostra el temps que triga el càlcul.
    
    Paràmetres:
        N (int): el nombre del qual volem comprovar la primeritat.
    
    Retorna:
        bool: True si és primer, False altrament.
    """
    start = time.time()  # Inici del comptador de temps

    # Casos trivials
    if N <= 1:
        print(f"{N} no és primer (per definició).")
        return False
    if N <= 3:
        print(f"{N} és primer (cas base).")
        return True
    if N % 2 == 0:
        print(f"{N} no és primer (és parell).")
        return False

    # Busquem divisors fins a l'arrel quadrada de N
    # Cost: O(√N)
    limit = int(math.sqrt(N)) + 1
    for i in range(3, limit, 2):  # Només comprovem nombres senars
        if N % i == 0:
            print(f"{N} no és primer (divisible per {i}).")
            print(f"Temps: {time.time() - start:.6f} s")
            return False

    # Si no s’ha trobat cap divisor, és primer
    print(f"{N} és primer.")
    print(f"Temps: {time.time() - start:.6f} s")
    return True

    # -------------------------------
    # Complexitat temporal: O(√N)
    # Complexitat espacial: O(1)
    # -------------------------------



def fermatp(N: int, bases: list[int] = [2, 3, 5]) -> bool:
    """
    Comprova si un nombre N>10 és probablement primer mitjançant la tècnica de Fermat.
    Mostra el temps que triga el càlcul.

    Paràmetres:
        N (int): el nombre del qual volem comprovar la primeritat.
        bases (list[int]): les bases amb què el volem provar.
    
    Retorna:
        bool: True si és probablement primer, False si és compost o N ≤ 10.
    """
    start = time.time()  # Inici del comptador de temps

    # Condicions inicials
    if N <= 10:
        print(f"{N} ≤ 10 → no aplicable al test de Fermat.")
        return False
    if N % 2 == 0:
        print(f"{N} no és primer (és parell).")
        return False

    # Comprovem la condició de Fermat per a cada base
    # Fermat: a^(N-1) ≡ 1 (mod N)
    # Si alguna base no compleix, N és compost.
    for a in bases:
        if pow(a, N - 1, N) != 1:
            print(f"{N} no passa el test de Fermat per a la base {a}.")
            print(f"Temps: {time.time() - start:.6f} s")
            return False

    # Si passa totes les proves, és "probablement primer"
    print(f"{N} passa el test de Fermat per les bases {bases}.")
    print(f"Temps: {time.time() - start:.6f} s")
    return True

    # -------------------------------
    # Complexitat temporal: O(k · log N)
    #   - k = nombre de bases provades
    #   - cada pow() és O(log N) amb exponenciació modular ràpida
    # Complexitat espacial: O(1)
    # -------------------------------


In [8]:
assert factorp(12) == False
assert fermatp(12) == False

assert factorp(53) == True
assert fermatp(53) == True

pseudoprimer = 1729 # 7 * 13 * 19
assert factorp(pseudoprimer) == False
assert fermatp(pseudoprimer) == True # Mal clasificat! Tenim que extendre el nombre d'elements de la base
assert fermatp(pseudoprimer,[2,3,5,7]) == False # Ara si que ho fa bé

assert factorp(9973) == True
assert fermatp(9973) == True

assert factorp(2265091) == False # 2265091 = 2017*1123 que son primers molt grans i la factorització és lenta
assert fermatp(2265091) == False

12 no és primer (és parell).
12 no és primer (és parell).
53 és primer.
Temps: 0.000037 s
53 passa el test de Fermat per les bases [2, 3, 5].
Temps: 0.000017 s
1729 no és primer (divisible per 7).
Temps: 0.000019 s
1729 passa el test de Fermat per les bases [2, 3, 5].
Temps: 0.000016 s
1729 no passa el test de Fermat per a la base 7.
Temps: 0.000014 s
9973 és primer.
Temps: 0.000012 s
9973 passa el test de Fermat per les bases [2, 3, 5].
Temps: 0.000020 s
2265091 no és primer (divisible per 1123).
Temps: 0.000045 s
2265091 no passa el test de Fermat per a la base 2.
Temps: 0.000010 s


### 📊 Càlcul complexitat

Repeteix el codi de la teva funció i detalla els passos computacionals i l'ordre de complexitat amb O gran. 

In [ ]:
import math
import time


def factorp(N: int) -> bool:
    """
    Comprova si un determinat nombre és primer mitjançant la tècnica de factorització.
    Mostra el temps que triga el càlcul.
    
    Paràmetres:
        N (int): el nombre del qual volem comprovar la primeritat.
    
    Retorna:
        bool: True si és primer, False altrament.
    """
    start = time.time()  # Inici del comptador de temps

    # Casos trivials:
    if N <= 1:
        # 0 i 1 no són primers
        print(f"{N} no és primer (per definició).")
        return False
    if N <= 3:
        # 2 i 3 són primers
        print(f"{N} és primer (cas base).")
        return True
    if N % 2 == 0:
        # Els nombres parells > 2 no són primers
        print(f"{N} no és primer (és parell).")
        return False

    # Busquem divisors fins a l'arrel quadrada de N
    # Això redueix dràsticament el nombre d’iteracions
    limit = int(math.sqrt(N)) + 1

    for i in range(3, limit, 2):  # Només comprovem divisors senars
        if N % i == 0:
            # Si trobem un divisor, N no és primer
            print(f"{N} no és primer (divisible per {i}).")
            print(f"Temps: {time.time() - start:.6f} s")
            return False

    # Si no s’ha trobat cap divisor, N és primer
    print(f"{N} és primer.")
    print(f"Temps: {time.time() - start:.6f} s")
    return True

    # --------------------------------------------
    # Complexitat temporal: O(√N)
    #   - Es comproven com a màxim √N possibles divisors.
    # Complexitat espacial: O(1)
    #   - No empra estructures addicionals de dades.
    # --------------------------------------------



def fermatp(N: int, bases: list[int] = [2, 3, 5]) -> bool:
    """
    Comprova si un nombre N>10 és probablement primer mitjançant la tècnica de Fermat.
    Mostra el temps que triga el càlcul.

    Paràmetres:
        N (int): el nombre del qual volem comprovar la primeritat.
        bases (list[int]): les bases amb què el volem provar.
    
    Retorna:
        bool: True si és probablement primer, False si és compost o N ≤ 10.
    """
    start = time.time()  # Inici del comptador de temps

    # Condicions inicials
    if N <= 10:
        # El test de Fermat no s’aplica a nombres petits
        print(f"{N} ≤ 10 → no aplicable al test de Fermat.")
        return False
    if N % 2 == 0:
        # Els nombres parells > 2 no són primers
        print(f"{N} no és primer (és parell).")
        return False

    # Test de Fermat:
    # Si N és primer → a^(N-1) ≡ 1 (mod N) per totes les bases a coprimes amb N
    for a in bases:
        if pow(a, N - 1, N) != 1:
            # Si la congruència no es compleix, N és compost
            print(f"{N} no passa el test de Fermat per a la base {a}.")
            print(f"Temps: {time.time() - start:.6f} s")
            return False

    # Si passa totes les proves, és "probablement primer"
    print(f"{N} passa el test de Fermat per les bases {bases}.")
    print(f"Temps: {time.time() - start:.6f} s")
    return True

    # --------------------------------------------
    #  Complexitat temporal: O(k · log N)
    #   - k = nombre de bases provades
    #   - cada pow() empra exponenciació modular ràpida (O(log N))
    # Complexitat espacial: O(1)
    #   - només variables simples (N, a, bases)
    # --------------------------------------------
